# ATAC-seq Module1: Preprocessing and Quality Control

<img src="https://storage.googleapis.com/unmc_atac_data_examples/Tutorial1/images/ATACseqWorkflowLesson1.jpg" alt="Drawing" style="width: 1000px;"/>

## Overview & Purpose
This short tutorial demonstrates the intial processing steps for ATAC-seq analysis. In this module we focus on generating quality reports of the fastq files, adapter trimming, mapping, and removal of PCR duplicates.

In this tutorial we will process a randomly chosen published dataset. This is available from GEO: GSE67382
Bao X, Rubin AJ, Qu K, Zhang J et al. A novel ATAC-seq approach reveals lineage-specific reinforcement of the open chromatin landscape via cooperation between BAF and p63. Genome Biol 2015 Dec 18;16:284. PMID: 26683334

This dataset is paired-end 50 bp sequencing. We will analyze two samples representing NHEK cells with BAF depletion compared to a control. Note that to allow faster processing we have limited the reads to that of chromosome 4. 
  
### Required Files
In this stage of the module you will use the fastq files that have been prepared. In step1 we will copy these files over to your instance. You can also use this module on your own data or any published ATAC-seq dataset. 

<div class="alert-info" style="font-size:200%">
STEP1: Setup Environment
</div>

Initial items to configure your google cloud environment. In this step we will use conda to install the following packages:

Quality Reporting:
[fastqc](https://anaconda.org/bioconda/fastqc), [multiqc](https://anaconda.org/bioconda/multiqc)

Read Trimming: 
[trimmomatic](https://anaconda.org/bioconda/trimmomatic)

Mapping:
[bowtie2](https://anaconda.org/bioconda/bowtie2)

Deduplication:
[samtools](https://anaconda.org/bioconda/samtools), [picard](https://anaconda.org/bioconda/picard)

In [25]:
#!python -m ipykernel install --user --name ATACtraining
numthreads=!lscpu | grep '^CPU(s)'| awk '{print $2-1}'
numthreadsint = int(numthreads[0])
!conda config --prepend channels bioconda
!conda install -y -c bioconda fastqc bowtie2 picard multiqc samtools trimmomatic
!pip install jupyterquiz jupytercards
from jupyterquiz import display_quiz
from IPython.display import IFrame
from IPython.display import display
from jupytercards import display_flashcards
import pandas as pd

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - bowtie2
    - fastqc
    - multiqc
    - picard
    - samtools
    - trimmomatic


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1q             |       h166bdaf_1         2.1 MB  conda-forge
    picard-2.27.4              |       hdfd78af_0        15.4 MB  bioconda
    ------------------------------------------------------------
                                           Total:        17.5 MB

The following packages will be UPDATED:

  openssl                                 1.1.1q-h166bdaf_0 --> 1.1.1q-h166bdaf_1 None
  picard                                          2.18.29-0 --> 2.27.4-hdfd78af_0 None



picard-2.27.4        | 15.4 MB   | ##################################### | 100% 
openssl-1.1.1q       | 2.1 MB    | ##################################### | 1

## Setup FileSystem
Now lets create some folders to stay organized and copy over our prepared fastq files. We're going to create a directory called "Tutorial1" which we'll use for this module. We'll then create subfolders for our InputFiles and for the files that we'll be creating during this module. We'll also copy over the fasta file for chromosome 4 as well as some bowtie2 index files (don't worry we'll teach you how to create these index files).

In [26]:
#These commands create our directory structure.
#!cd $HOMEDIR
#!mkdir -p Tutorial1
#!mkdir -p Tutorial1/InputFiles
!mkdir -p Tutorial1/QC
!mkdir -p Tutorial1/Trimmed
!mkdir -p Tutorial1/Mapped
#!mkdir -p Tutorial1/RefGenome
#!mkdir -p Tutorial1/LessonImages
#!echo $PWD

#These commands help identify the google cloud storage bucket where the example files are held.
project_id = "nosi-unmc-seq"
original_bucket = "gs://unmc_atac_data_examples/Tutorial1"
#!gsutil -m cp $original_bucket/images/* Tutorial1/LessonImages
#This command copies our example files to the Tutorial1/Inputfiles folder that we created above.
! gsutil -m cp $original_bucket/InputFiles/*fastq.gz Tutorial1/InputFiles
#! gsutil -m cp $original_bucket/InputFiles/hg38* Tutorial1/RefGenome


Copying gs://unmc_atac_data_examples/Tutorial1/InputFiles/CTL_R1.fastq.gz...
Copying gs://unmc_atac_data_examples/Tutorial1/InputFiles/CTL_R2.fastq.gz...    
Copying gs://unmc_atac_data_examples/Tutorial1/InputFiles/Mutant_R1.fastq.gz... 
Copying gs://unmc_atac_data_examples/Tutorial1/InputFiles/Mutant_R2.fastq.gz... 
/ [4/4 files][124.4 MiB/124.4 MiB] 100% Done                                    
Operation completed over 4 objects/124.4 MiB.                                    



### OK
Let's make sure that the files copied correctly. You should see 4 files after running the following command:

In [27]:
!ls Tutorial1/InputFiles


CTL_R1.fastq.gz  CTL_R2.fastq.gz  Mutant_R1.fastq.gz  Mutant_R2.fastq.gz


<div class="alert-info" style="font-size:200%">
STEP2: QC
</div>

Sequences are typically provided as files in fastq format. This format includes 4 lines per sequence.


In [28]:
display_flashcards('Tutorial1/LessonImages/FastqFlashCard.json')

<IPython.core.display.Javascript object>

### Click on the above image to see what each line represents.

Next, let's take a look at the sequence quality of the raw reads usinq fastqc:

In [29]:
#This command runs fastqc on each fastq.gz file inside our InputFiles directory and stores the ouput reports in our QC directory.
!fastqc -t $numthreadsint -q -o Tutorial1/QC Tutorial1/InputFiles/*fastq.gz

#We then use multiqc to summarize the report.
!multiqc -o Tutorial1/QC -f Tutorial1/QC 2> Tutorial1/QC/multiqc_log.txt

#We'll load this into a pandas table to work in this context, but fastqc also produces an html report that you can browse.
dframe = pd.read_csv("Tutorial1/QC/multiqc_data/multiqc_fastqc.txt", sep='\t')
display(dframe)

,Sample,Filename,File type,Encoding,Total Sequences,Sequences flagged as poor quality,Sequence length,%GC,total_deduplicated_percentage,avg_sequence_length,basic_statistics,per_base_sequence_quality,per_sequence_quality_scores,per_base_sequence_content,per_sequence_gc_content,per_base_n_content,sequence_length_distribution,sequence_duplication_levels,overrepresented_sequences,adapter_content
0,CTL_R1,CTL_R1.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,721311.0,0.0,50.0,43.0,34.116497,50.0,pass,pass,pass,fail,pass,pass,pass,fail,warn,pass
1,CTL_R2,CTL_R2.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,721311.0,0.0,50.0,42.0,33.459856,50.0,pass,pass,pass,fail,warn,pass,pass,fail,fail,pass
2,Mutant_R1,Mutant_R1.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,848511.0,0.0,50.0,42.0,41.516085,50.0,pass,pass,pass,fail,pass,pass,pass,fail,warn,pass
3,Mutant_R2,Mutant_R2.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,848511.0,0.0,50.0,42.0,37.731878,50.0,pass,pass,pass,fail,warn,pass,pass,fail,fail,pass


Alternatively, we can view the fastqc html files:

In [30]:
#We can display the resulting fastqc results.
IFrame(src='Tutorial1/QC/CTL_R1_fastqc.html', width=1080, height=800)

Look at the the "Per base seqeuence content" in the above FastQC report. We'll trim the reads to remove some of this effect. For now, think about possible explanations for this result.

Also look at the "Sequence Duplication Levels". Sometimes duplicates appear due to the PCR amplification step of library preparation. We'll remove duplicates in a later step. 

Lastly, look at the report at the "Overrepresented sequences". What are some possible explanations for this result?

<div class="alert-info" style="font-size:120%">
Trimming
</div>
Next let's trim our sequences.

Why is it particularly important to trim the reads in ATAC-seq? To understand let's review how ATAC-seq works. Tn5 inserts adapter sequences into accessible regions. 

<img src="Tutorial1/LessonImages/adapterinsert.jpg" alt="Drawing" style="width: 400px;"/>

Image source: [Grandi et al., Nature Protocols 2022](https://www.nature.com/articles/s41596-022-00692-9)


What would happen if the distance between inserted sites is short? For example our sequencing lenghth in the example dataset is 50 bp, so what would the sequence look like if our fragment (insert size) is only 30 bp long?  

<div class="alert-info" style="font-size:200%">
Interactive Quiz Question 1: Click on the correct answer in following cell.
</div>

In [31]:
display_quiz("Tutorial1/LessonImages/adapterQuiz.json")

<IPython.core.display.Javascript object>

## Let's use trimmomatic to prepare the sequeces before mapping.

In [34]:
#This will trim off N's as well as nextera adapters present in ATAC-seq library preparation. placing the trimmed reads in our Trimmed folder.
!trimmomatic PE -threads $numthreadsint Tutorial1/InputFiles/CTL_R1.fastq.gz Tutorial1/InputFiles/CTL_R2.fastq.gz Tutorial1/Trimmed/CTLtrimmed_R1.fastq.gz Tutorial1/Trimmed/CTLunpaired_R1.fastq.gz Tutorial1/Trimmed/CTLtrimmed_R2.fastq.gz Tutorial1/Trimmed/CTLunpaired_R2.fastq.gz ILLUMINACLIP:Tutorial1/RefGenome/NexteraPE.fa:2:30:10 LEADING:3 TRAILING:3

TrimmomaticPE: Started with arguments:
 -threads 3 Tutorial1/InputFiles/CTL_R1.fastq.gz Tutorial1/InputFiles/CTL_R2.fastq.gz Tutorial1/Trimmed/CTLtrimmed_R1.fastq.gz Tutorial1/Trimmed/CTLunpaired_R1.fastq.gz Tutorial1/Trimmed/CTLtrimmed_R2.fastq.gz Tutorial1/Trimmed/CTLunpaired_R2.fastq.gz ILLUMINACLIP:Tutorial1/RefGenome/NexteraPE.fa:2:30:10 LEADING:3 TRAILING:3
Using PrefixPair: 'AGATGTGTATAAGAGACAG' and 'AGATGTGTATAAGAGACAG'
Using Long Clipping Sequence: 'GTCTCGTGGGCTCGGAGATGTGTATAAGAGACAG'
Using Long Clipping Sequence: 'TCGTCGGCAGCGTCAGATGTGTATAAGAGACAG'
Using Long Clipping Sequence: 'CTGTCTCTTATACACATCTCCGAGCCCACGAGAC'
Using Long Clipping Sequence: 'CTGTCTCTTATACACATCTGACGCTGCCGACGA'
ILLUMINACLIP: Using 1 prefix pairs, 4 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Quality encoding detected as phred33
Input Read Pairs: 721311 Both Surviving: 721308 (100.00%) Forward Only Surviving: 0 (0.00%) Reverse Only Surviving: 2 (0.00%) Dropped: 1 (0.00%)
Trim

## Let's do this for the other sample as well.

In [35]:
#This will trim off N's as well as nextera adapters present in ATAC-seq library preparation. placing the trimmed reads in our Trimmed folder.
!trimmomatic PE -threads $numthreadsint Tutorial1/InputFiles/Mutant_R1.fastq.gz Tutorial1/InputFiles/Mutant_R2.fastq.gz Tutorial1/Trimmed/Mutanttrimmed_R1.fastq.gz Tutorial1/Trimmed/Mutantunpaired_R1.fastq.gz Tutorial1/Trimmed/Mutanttrimmed_R2.fastq.gz Tutorial1/Trimmed/Mutantunpaired_R2.fastq.gz ILLUMINACLIP:Tutorial1/RefGenome/NexteraPE.fa:2:30:10 LEADING:3 TRAILING:3

TrimmomaticPE: Started with arguments:
 -threads 3 Tutorial1/InputFiles/Mutant_R1.fastq.gz Tutorial1/InputFiles/Mutant_R2.fastq.gz Tutorial1/Trimmed/Mutanttrimmed_R1.fastq.gz Tutorial1/Trimmed/Mutantunpaired_R1.fastq.gz Tutorial1/Trimmed/Mutanttrimmed_R2.fastq.gz Tutorial1/Trimmed/Mutantunpaired_R2.fastq.gz ILLUMINACLIP:Tutorial1/RefGenome/NexteraPE.fa:2:30:10 LEADING:3 TRAILING:3
Using PrefixPair: 'AGATGTGTATAAGAGACAG' and 'AGATGTGTATAAGAGACAG'
Using Long Clipping Sequence: 'GTCTCGTGGGCTCGGAGATGTGTATAAGAGACAG'
Using Long Clipping Sequence: 'TCGTCGGCAGCGTCAGATGTGTATAAGAGACAG'
Using Long Clipping Sequence: 'CTGTCTCTTATACACATCTCCGAGCCCACGAGAC'
Using Long Clipping Sequence: 'CTGTCTCTTATACACATCTGACGCTGCCGACGA'
ILLUMINACLIP: Using 1 prefix pairs, 4 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Quality encoding detected as phred33
Input Read Pairs: 848511 Both Surviving: 848499 (100.00%) Forward Only Surviving: 0 (0.00%) Reverse Only Surviving: 1 (0.00%) Dropp

## Now let's summarize the trimming results.

In [36]:
!fastqc -t $numthreadsint -q -o Tutorial1/Trimmed Tutorial1/Trimmed/*fastq.gz
!multiqc -o Tutorial1/QC -f Tutorial1/Trimmed 2> Tutorial1/QC/multiqc_log.txt

dframe = pd.read_csv("Tutorial1/QC/multiqc_data/multiqc_general_stats.txt", sep='\t')
display(dframe)


,Sample,FastQC_mqc-generalstats-fastqc-percent_duplicates,FastQC_mqc-generalstats-fastqc-percent_gc,FastQC_mqc-generalstats-fastqc-avg_sequence_length,FastQC_mqc-generalstats-fastqc-percent_fails,FastQC_mqc-generalstats-fastqc-total_sequences
0,CTLtrimmed_R1,65.883549,43.0,49.998468,20.0,721308.0
1,CTLtrimmed_R2,66.540191,42.0,49.999289,30.0,721308.0
2,CTLunpaired_R1,0.000000,0.0,0.000000,0.0,0.0
3,CTLunpaired_R2,0.000000,45.0,50.000000,40.0,2.0
4,Mutanttrimmed_R1,58.484596,42.0,49.996565,20.0,848499.0
5,Mutanttrimmed_R2,62.268044,42.0,49.996758,30.0,848499.0
6,Mutantunpaired_R1,0.000000,0.0,0.000000,0.0,0.0
7,Mutantunpaired_R2,0.000000,74.0,50.000000,30.0,1.0


<div class="alert-info" style="font-size:200%">
Step3: Mapping
</div>
Our fastq files include sequences and quality scores for each base, but we want to figure out which genomic location these sequences came from. To do this we will map each sequence to a reference genome using bowtie2. 
 

Mapping reads requires a reference genome. Due to time and memory considerations, in this tutorial we  prepared that file for you and will only map to chr4. However, in a full analysis, we would map to the entire genome. To do so you would need a fasta file corresponding to the reference genome (e.g. [hg38.fa](https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/)) from which you'd create an index of the genome using bowtie2-build. This can be done with the command: 

bowtie2-build reference_genome_file.fa outputprefix.

As mentioned, we've gone ahead and created the index for you, and, earlier, you copied them into the RefGenome directory. These index files end in the bt2 extension. 



In [37]:
!ls Tutorial1/RefGenome/*bt2

Tutorial1/RefGenome/hg38chr4.1.bt2  Tutorial1/RefGenome/hg38chr4.4.bt2
Tutorial1/RefGenome/hg38chr4.2.bt2  Tutorial1/RefGenome/hg38chr4.rev.1.bt2
Tutorial1/RefGenome/hg38chr4.3.bt2  Tutorial1/RefGenome/hg38chr4.rev.2.bt2


These index files were created from our fasta file:

In [38]:
!ls Tutorial1/RefGenome/*fa

Tutorial1/RefGenome/NexteraPE.fa  Tutorial1/RefGenome/hg38chr4.fa


Notice that the single fasta file created mutiple index files. When we align we'll specify the prefix of the index files.

In [39]:
#Notes: The -x option specifies the prefix of the index. -1 specifies our left-end trimmed reads file. -2 specifies our right-end trimmed reads file. -S specifies our output file in sam format.
!bowtie2 -p $numthreadsint -x Tutorial1/RefGenome/hg38chr4 -1 Tutorial1/Trimmed/CTLtrimmed_R1.fastq.gz -2 Tutorial1/Trimmed/CTLtrimmed_R2.fastq.gz -S Tutorial1/Mapped/CTL.sam


721308 reads; of these:
  721308 (100.00%) were paired; of these:
    291053 (40.35%) aligned concordantly 0 times
    362493 (50.25%) aligned concordantly exactly 1 time
    67762 (9.39%) aligned concordantly >1 times
    ----
    291053 pairs aligned concordantly 0 times; of these:
      60279 (20.71%) aligned discordantly 1 time
    ----
    230774 pairs aligned 0 times concordantly or discordantly; of these:
      461548 mates make up the pairs; of these:
        415122 (89.94%) aligned 0 times
        19801 (4.29%) aligned exactly 1 time
        26625 (5.77%) aligned >1 times
71.22% overall alignment rate


In [40]:
##Let's do the same thing for our other sample.
!bowtie2 -p $numthreadsint -x Tutorial1/RefGenome/hg38chr4 -1 Tutorial1/Trimmed/Mutanttrimmed_R1.fastq.gz -2 Tutorial1/Trimmed/Mutanttrimmed_R2.fastq.gz -S Tutorial1/Mapped/Mutant.sam

848499 reads; of these:
  848499 (100.00%) were paired; of these:
    255742 (30.14%) aligned concordantly 0 times
    486793 (57.37%) aligned concordantly exactly 1 time
    105964 (12.49%) aligned concordantly >1 times
    ----
    255742 pairs aligned concordantly 0 times; of these:
      73634 (28.79%) aligned discordantly 1 time
    ----
    182108 pairs aligned 0 times concordantly or discordantly; of these:
      364216 mates make up the pairs; of these:
        306498 (84.15%) aligned 0 times
        19708 (5.41%) aligned exactly 1 time
        38010 (10.44%) aligned >1 times
81.94% overall alignment rate


### Answer the following question only if you are using the example dataset we provided. This question is simply a check to ensure everything was processed correctly.

In [41]:
display_flashcards('Tutorial1/LessonImages/alignment.json')

<IPython.core.display.Javascript object>

## Bowtie2 output a file in [sam format](https://samtools.github.io/hts-specs/SAMv1.pdf) which contains the original sequence, quality scores, and the genomic coordinates matching each read. 

In the next commands we'll convert the file to the more compressed [bam format](https://genome.ucsc.edu/goldenPath/help/bam.html) and sort the reads by chromosomal coordinates.

In [42]:
#This will convert to bam by using samtools view with the -b option. The h and S option tells samtools that the file has a header and is in sam format. We will pipe this to samtools sort. Pay attention to the "-" at the end of the sort command which tells samtools to use stdin.
!samtools view -q 10 -bhS Tutorial1/Mapped/CTL.sam | samtools sort -o Tutorial1/Mapped/CTL.bam - 
print("done")

done


In [43]:
#Let's do the same thing for our Mutant sample.
!samtools view -q 10 -bhS Tutorial1/Mapped/Mutant.sam | samtools sort -o Tutorial1/Mapped/Mutant.bam - 
print("done")

done


You may have noticed the parameters -bhS and -q 10 in the above commands. Briefly, -bhS describes aspects of the file to samtools, such that you want to output a bam file (the b option), that it has a header (the h option), and that it is currently in sam format (the S option). We also specified -q 10 which removes reads with a mapping score <= 10. 

<div class="alert-info" style="font-size:200%">
Interactive Quiz Question 2: Click on the correct answer in the following cell.
</div>

In [44]:
display_quiz("Tutorial1/LessonImages/mappingquality.json")

<IPython.core.display.Javascript object>

<div class="alert-info" style="font-size:200%">
Step4: Removal of Duplicates
</div>
It's important to remove duplicates from our reads because part of the ATAC-seq method includes a PCR step for library amplification. This can create biases in the data resulting from PCR duplicates. To understand how PCR duplicates can affect the analysis, let's jump ahead a bit. Accessibile sites are represented by ATAC-seq "peaks" of signal.

<img src="Tutorial1/LessonImages/PeaksExample.jpg" alt="Drawing" style="width: 200px;"/>

<div class="alert-info" style="font-size:200%">
Interactive Quiz Question 3: Click on the correct answer in the following cell.
</div>

In [45]:
display_quiz("Tutorial1/LessonImages/duplicateQuiz.json")

<IPython.core.display.Javascript object>

Okay, let's remove these duplicates using Picard.

In [46]:
#this will take the sorted bam file and remove duplicates, saving a new bam file and a summary in a text file.
!picard MarkDuplicates --REMOVE_DUPLICATES TRUE -I Tutorial1/Mapped/CTL.bam -O Tutorial1/Mapped/CTL_dedup.bam --METRICS_FILE Tutorial1/Mapped/CTL_dedup_metrics.txt --QUIET 2> Tutorial1/Mapped/PicardLog.txt
print("done")

done


In [47]:
#We also should do this for the other sample.
!picard MarkDuplicates --REMOVE_DUPLICATES TRUE -I Tutorial1/Mapped/Mutant.bam -O Tutorial1/Mapped/Mutant_dedup.bam --METRICS_FILE Tutorial1/Mapped/Mutant_dedup_metrics.txt --QUIET 2> Tutorial1/Mapped/PicardLog2.txt
print("done")

done


In [48]:
#We can use multiqc to summarize the metrics
!multiqc -o Tutorial1/QC -f Tutorial1/Mapped 2> Tutorial1/Mapped/multiqc_log.txt
dframe = pd.read_csv("Tutorial1/QC/multiqc_data/multiqc_general_stats.txt", sep='\t')
display(dframe)

,Sample,Picard_mqc-generalstats-picard-PERCENT_DUPLICATION
0,CTL,0.557463
1,Mutant,0.585257


<div class="alert-success" style="font-size:200%">
Great job! 
</div>
We have completed the preprocessing steps and are ready to move on to some downstream analysis. Take a break here or move on to the next tutorial: 

[Visualization and Peak Detection](./ATACseq_Tutorial2_PeakDetection.ipynb). 
